# Processamento de Linguagem Natural - Bag of Words - Prática 02

## Bag of Words


O modelo de "saco de palavras" é uma representação simplificada usada no processamento de linguagem natural e recuperação de informação. Neste modelo, um texto (como uma sentença ou um documento) é representado como o saco (multiset) de suas palavras, desconsiderando a gramática e até a ordem das palavras, mas mantendo a multiplicidade.

Na classificação de documentos, um saco de palavras é um vetor esparso de ocorrência de contagens de palavras; Ou seja, um histograma esparso sobre o vocabulário.

### Carregando um Dataset de um Site de E-commerce (em português)

In [5]:
import gzip
import json
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Carregando o dataset
corpus = list()
with gzip.open('ecommerce.json.gz') as fp:
    for line in fp:
        entry = line.decode('utf8')
        corpus.append(json.loads(entry))

In [3]:
from pprint import pprint
pprint(corpus[0])

{'_id': 120008322,
 'cat': ' Automotivo',
 'descr': 'Chegou o kit que junta resistência e conforto, além de níveis '
          'máximos de segurança. São 4 pneus para seu carro ficar completo e '
          'com a qualificação que você precisa.\n'
          'Com os conhecimentos avançados de hoje e um entusiasmo pela '
          'direção, os engenheiros da Pirelli puderam dar grandes passos. Cada '
          'pneu da Pirelli é responsável não só pelo desempenho, mas também '
          'por uma "vontade de ir pra estrada", comunicando-se com o motorista '
          'e gerando um melhor entendimento do desempenho do veículo, ou seja, '
          'a Pirelli transforma a sua viagem em uma aventura divertida e livre '
          'de problemas. Pneu Pirelli para carros com rodas aro 16, modelo '
          'high performance Phanthon, perfil baixo proporcionando maior '
          'estabilidade nas curvas, excelente qualidade e durabilidade para '
          'pistas.\n'
          '\n'
          'I

In [4]:
print (corpus[0]['descr'])

Chegou o kit que junta resistência e conforto, além de níveis máximos de segurança. São 4 pneus para seu carro ficar completo e com a qualificação que você precisa.
Com os conhecimentos avançados de hoje e um entusiasmo pela direção, os engenheiros da Pirelli puderam dar grandes passos. Cada pneu da Pirelli é responsável não só pelo desempenho, mas também por uma "vontade de ir pra estrada", comunicando-se com o motorista e gerando um melhor entendimento do desempenho do veículo, ou seja, a Pirelli transforma a sua viagem em uma aventura divertida e livre de problemas. Pneu Pirelli para carros com rodas aro 16, modelo high performance Phanthon, perfil baixo proporcionando maior estabilidade nas curvas, excelente qualidade e durabilidade para pistas.

Imagens meramente ilustrativas.
Todas as informações divulgadas são de responsabilidade do fabricante/fornecedor.


## Construindo um classificador

In [7]:
produto = corpus[1]['name']
categoria = corpus[1]['cat']
print(produto, '---', categoria)

Chandon Brut Rosé 750 ml ---  Alimentos e Bebidas


In [8]:
# Construindo um classificador para produtos e categorias (considerando apenas os 10 mil primeiros produtos)
dataset = list()
for entry in corpus[:10000]:
    if 'cat' in entry:
        dataset.append( (entry['name'], entry['cat'].lower().strip()) )

In [9]:
len(dataset)

9953

In [12]:
pprint(dataset[:10])

[('Kit com 4 Pneus de Alta Performance Pirelli Aro 16 205/55R16 Phantom',
  'automotivo'),
 ('Chandon Brut Rosé 750 ml', 'alimentos e bebidas'),
 ('Kit com 2 Vodkas Sueca Absolut Vanilia 1000ml', 'alimentos e bebidas'),
 ('Kit  - Livros de Colorir: Jardim Secreto + Floresta Encantada + Reino '
  'Animal',
  'livros'),
 ("Livro - Assassin's Creed: Submundo", 'livros'),
 ('BCAA 2400 - 100 Cápsulas - Nitech Nutrition', 'suplementos e vitaminas'),
 ('100% Whey - 900g - Baunilha - Nitech Nutrition', 'suplementos e vitaminas'),
 ('Whey Protein Isolate - 900g - Morango - Nitech Nutrition',
  'suplementos e vitaminas'),
 ('100% Whey - 900g - Chocolate - Nitech Nutrition', 'suplementos e vitaminas'),
 ('BCAA 2400 - 200 Cápsulas - Nitech Nutrition', 'suplementos e vitaminas')]


In [13]:
# Quantas categorias distintas nós temos e quantos itens por categoria?
from collections import Counter
counter = Counter([cat for prod, cat in dataset])
pprint(counter.most_common())

[('bebês', 1208),
 ('eletroportáteis', 1052),
 ('automotivo', 915),
 ('utilidades domésticas', 857),
 ('suplementos e vitaminas', 787),
 ('ar-condicionado e aquecedores', 754),
 ('informática', 706),
 ('cama, mesa e banho', 670),
 ('tv e home theater', 644),
 ('perfumaria', 532),
 ('beleza e saúde', 497),
 ('dvds e blu-ray', 433),
 ('relógios', 410),
 ('pet shop', 391),
 ('instrumentos musicais', 44),
 ('celulares e telefones', 18),
 ('eletrodomésticos', 16),
 ('áudio', 13),
 ('alimentos e bebidas', 2),
 ('livros', 2),
 ('brinquedos', 1),
 ('linha industrial', 1)]


# Construindo um Classificador SVM com Bag of Words

http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
import nltk
stopwords = nltk.corpus.stopwords.words('portuguese')

In [16]:
# Construindo o modelo SVM com Pipeline
modelo = Pipeline([('vect', TfidfVectorizer()), ('clf', SVC(kernel = 'linear', probability = True))])

In [17]:
?LabelEncoder

In [18]:
# Objeto para Normalização dos labels
encoder = LabelEncoder()

In [19]:
# Obtendo dados e labels
data = [prod for prod, cat in dataset]
labels = [cat for prod, cat in dataset]
len(data)

9953

In [20]:
data[:3]

['Kit com 4 Pneus de Alta Performance Pirelli Aro 16 205/55R16 Phantom',
 'Chandon Brut Rosé 750 ml',
 'Kit com 2 Vodkas Sueca Absolut Vanilia 1000ml']

In [21]:
labels[:3]

['automotivo', 'alimentos e bebidas', 'alimentos e bebidas']

In [24]:
# Normalização dos labels (converter de string para número)
target = encoder.fit_transform(labels)
target[:3]

array([2, 0, 0])

In [27]:
# Items
print(encoder.classes_.item(2))
print(encoder.classes_.item(0))

automotivo
alimentos e bebidas


In [28]:
# Fit do modelo
modelo.fit(data, target)

Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [29]:
# Prevendo a categoria a partir da descrição
modelo.predict(["Refrigerador Brastemp com função frostfree"])

array([9])

In [30]:
# Prevendo a categoria a partir da descrição
print (encoder.classes_[9])

eletrodomésticos


In [31]:
# Prevendo a categoria a partir da descrição
print(encoder.classes_[ modelo.predict(["baixo com 5 cordas"])])

['instrumentos musicais']


In [32]:
# Prevendo a categoria a partir da descrição
print(encoder.classes_[ modelo.predict(["panela"])])

['utilidades domésticas']


In [33]:
# Probabilidades de um produto
probs = modelo.predict_proba(["Ventilador"])
probs

array([[  1.78337929e-08,   2.02699370e-08,   7.29496812e-08,
          1.72540865e-07,   6.96643553e-08,   1.10161254e-08,
          2.68772950e-08,   1.06455706e-08,   5.86833352e-08,
          1.90750682e-08,   9.99980329e-01,   8.75746680e-08,
          4.99291778e-07,   1.27312949e-05,   3.62266742e-07,
          9.71916328e-08,   1.94984037e-07,   1.36219961e-06,
          2.74377276e-07,   1.71003226e-07,   9.31987255e-08,
          3.31847787e-06]])

In [34]:
# Probabidades de categorias para o objeto Ventilador
guess = [(class_, probs.item(n)) for n, class_ in enumerate(encoder.classes_)]
pprint(guess)

[('alimentos e bebidas', 1.7833792933386225e-08),
 ('ar-condicionado e aquecedores', 2.0269937021380632e-08),
 ('automotivo', 7.294968117880753e-08),
 ('bebês', 1.7254086532506347e-07),
 ('beleza e saúde', 6.96643553086055e-08),
 ('brinquedos', 1.1016125351839872e-08),
 ('cama, mesa e banho', 2.6877295041868853e-08),
 ('celulares e telefones', 1.064557058071694e-08),
 ('dvds e blu-ray', 5.868333519196251e-08),
 ('eletrodomésticos', 1.9075068163111033e-08),
 ('eletroportáteis', 0.9999803285835196),
 ('informática', 8.757466798902202e-08),
 ('instrumentos musicais', 4.992917777117273e-07),
 ('linha industrial', 1.273129488623465e-05),
 ('livros', 3.6226674180255076e-07),
 ('perfumaria', 9.71916327528799e-08),
 ('pet shop', 1.9498403697302253e-07),
 ('relógios', 1.3621996109430775e-06),
 ('suplementos e vitaminas', 2.7437727627196557e-07),
 ('tv e home theater', 1.7100322579995142e-07),
 ('utilidades domésticas', 9.31987254530733e-08),
 ('áudio', 3.318477872003584e-06)]


In [35]:
# Probabidade ajustada de categorias para o objeto Ventilador
from operator import itemgetter
for cat, proba in sorted(guess, key = itemgetter(1), reverse = True):
    print ('{}: {:.4f}'.format(cat, proba))

eletroportáteis: 1.0000
linha industrial: 0.0000
áudio: 0.0000
relógios: 0.0000
instrumentos musicais: 0.0000
livros: 0.0000
suplementos e vitaminas: 0.0000
pet shop: 0.0000
bebês: 0.0000
tv e home theater: 0.0000
perfumaria: 0.0000
utilidades domésticas: 0.0000
informática: 0.0000
automotivo: 0.0000
beleza e saúde: 0.0000
dvds e blu-ray: 0.0000
cama, mesa e banho: 0.0000
ar-condicionado e aquecedores: 0.0000
eletrodomésticos: 0.0000
alimentos e bebidas: 0.0000
brinquedos: 0.0000
celulares e telefones: 0.0000


In [63]:
# exercício - Testar categorias a partir de produtos
# Prevendo a categoria a partir da descrição

produtos = ["baixo com 5 cordas", "notebook asus", "roda aro 15", "lg", "chocolate", "ração"]
for produto in produtos:
    print(produto, '==>' ,encoder.classes_[ modelo.predict([produto])])

baixo com 5 cordas ==> ['instrumentos musicais']
notebook asus ==> ['informática']
roda aro 15 ==> ['automotivo']
lg ==> ['tv e home theater']
chocolate ==> ['suplementos e vitaminas']
ração ==> ['pet shop']
